# Deploy TinyLlama-1.1B-Chat as SageMaker Serverless Endpoint

This notebook demonstrates how to deploy the **TinyLlama-1.1B-Chat-v1.0** model as a SageMaker Serverless Inference Endpoint using the HuggingFace Text Generation Inference (TGI) container.

## Overview

- **Model**: TinyLlama/TinyLlama-1.1B-Chat-v1.0
- **Endpoint Type**: Serverless (scales to zero when idle)
- **Memory**: 3072 MB (3 GB)
- **Max Concurrency**: 10
- **Cost**: Pay-per-use, ~$0.00002 per second of inference

## Benefits of Serverless Endpoints

- ✅ No charge when idle (scales to zero)
- ✅ Automatic scaling based on traffic
- ✅ No instance management
- ✅ Ideal for development and intermittent workloads
- ⚠️ Cold start latency (10-60 seconds when inactive)

## 1. Import Required Libraries

In [8]:
import boto3
import json
from datetime import datetime
import sagemaker

# Get versions safely
try:
    sm_version = sagemaker.__version__
except AttributeError:
    import importlib.metadata
    sm_version = importlib.metadata.version('sagemaker')

print(f"SageMaker SDK version: {sm_version}")
print(f"Boto3 version: {boto3.__version__}")

SageMaker SDK version: 3.4.0
Boto3 version: 1.42.44


## 2. Set Up AWS SageMaker Session

Initialize the SageMaker session and get the execution role. The role must have permissions to create SageMaker models and endpoints.

In [ ]:
# Initialize SageMaker session and boto3 clients
sess = sagemaker.Session()
region = sess.boto_region_name

# Get execution role - works in SageMaker environments or locally
try:
    role = sagemaker.get_execution_role()
except ValueError:
    # Running outside SageMaker, need to specify role ARN manually
    # Option 1: Get from existing SageMaker execution roles
    iam = boto3.client('iam')
    try:
        # Try to find an existing SageMaker execution role
        roles = iam.list_roles(PathPrefix='/service-role/')
        sagemaker_roles = [r for r in roles['Roles'] if 'SageMaker' in r['RoleName']]
        if sagemaker_roles:
            role = sagemaker_roles[0]['Arn']
            print(f"Using existing SageMaker role: {sagemaker_roles[0]['RoleName']}")
        else:
            raise Exception("No SageMaker execution role found. Please create one in IAM console.")
    except Exception as e:
        print(f"Error finding role: {e}")
        print("\nPlease specify your SageMaker execution role ARN manually:")
        print("role = 'arn:aws:iam::ACCOUNT_ID:role/service-role/YOUR-SAGEMAKER-ROLE'")
        raise

# Create SageMaker client for low-level API calls
sm_client = boto3.client('sagemaker', region_name=region)
sm_runtime = boto3.client('sagemaker-runtime', region_name=region)

print(f"SageMaker Execution Role: {role}")
print(f"AWS Region: {region}")

AttributeError: module 'sagemaker' has no attribute 'Session'

## 3. Define Model Configuration

Configure the TinyLlama model and TGI (Text Generation Inference) container settings.

In [ ]:
# Model configuration
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
endpoint_name = f"tinyllama-serverless-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

# HuggingFace TGI container configuration
hub = {
    'HF_MODEL_ID': model_id,
    'HF_TASK': 'text-generation',
    'MAX_INPUT_LENGTH': '2048',
    'MAX_TOTAL_TOKENS': '4096',
    'MAX_BATCH_PREFILL_TOKENS': '4096',
}

print(f"Model ID: {model_id}")
print(f"Endpoint Name: {endpoint_name}")
print(f"Container Config: {json.dumps(hub, indent=2)}")

## 4. Create HuggingFace Model

Create a HuggingFaceModel instance that will be deployed to SageMaker. This uses the HuggingFace TGI (Text Generation Inference) container optimized for LLM inference.

In [ ]:
# Get the HuggingFace TGI container image URI
image_uri = sagemaker.image_uris.retrieve(
    framework="huggingface",
    region=region,
    version="2.1.1",
    image_scope="inference",
    base_framework_version="pytorch2.1",
    py_version="py310",
    container_version="text-generation-inference2.0.1",
    instance_type="ml.g5.xlarge"  # Used for image URI retrieval only
)

# Create SageMaker model using boto3 client
model_name = f"tinyllama-model-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image_uri,
        'Environment': hub
    }
)

print("✓ SageMaker Model created successfully")
print(f"  Model Name: {model_name}")
print(f"  Image URI: {image_uri}")

## 5. Configure Serverless Endpoint

Configure the serverless inference settings:
- **Memory Size**: 3072 MB (3 GB) - Sufficient for TinyLlama 1.1B model
- **Max Concurrency**: 10 - Maximum concurrent invocations

In [ ]:
# Serverless configuration values
serverless_memory_mb = 3072  # 3 GB - sufficient for TinyLlama 1.1B
serverless_max_concurrency = 10  # Maximum concurrent invocations

print("✓ Serverless configuration set")
print(f"  Memory: {serverless_memory_mb} MB")
print(f"  Max Concurrency: {serverless_max_concurrency}")

## 6. Deploy Model to Serverless Endpoint

Deploy the model to a SageMaker serverless endpoint. This will:
1. Create the SageMaker model
2. Create the endpoint configuration with serverless settings
3. Create and deploy the endpoint

**Note**: Initial deployment takes 3-5 minutes for model download and container initialization.

In [ ]:
# Create endpoint configuration with serverless settings
endpoint_config_name = f"tinyllama-config-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'ServerlessConfig': {
                'MemorySizeInMB': serverless_memory_mb,
                'MaxConcurrency': serverless_max_concurrency
            }
        }
    ]
)

print(f"✓ Endpoint configuration created: {endpoint_config_name}")

# Deploy the endpoint
print(f"🚀 Deploying model to endpoint: {endpoint_name}")
print("⏳ This will take 3-5 minutes...")

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# Wait for endpoint to be in service
print("⏳ Waiting for endpoint to be in service...")
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

print(f"✅ Endpoint deployed successfully!")
print(f"   Endpoint name: {endpoint_name}")
print(f"   Endpoint type: Serverless")
print(f"   Status: InService")

## 7. Test the Endpoint

Send test prompts to the deployed endpoint and verify the model is working correctly.

In [ ]:
# Test with a simple prompt
test_prompt = "What is the capital of France?"

payload = {
    "inputs": test_prompt,
    "parameters": {
        "max_new_tokens": 128,
        "temperature": 0.7,
        "top_p": 0.9,
        "do_sample": True,
    }
}

print(f"📝 Prompt: {test_prompt}")
print("⏳ Generating response...")

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

result = json.loads(response['Body'].read())

print("\n✅ Response:")
print("=" * 60)
print(result[0]["generated_text"])
print("=" * 60)

In [ ]:
# Test with a coding question
coding_prompt = "Write a Python function to calculate the factorial of a number."

payload = {
    "inputs": coding_prompt,
    "parameters": {
        "max_new_tokens": 256,
        "temperature": 0.3,  # Lower temperature for more focused code generation
        "top_p": 0.95,
    }
}

print(f"📝 Prompt: {coding_prompt}")
print("⏳ Generating response...")

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

result = json.loads(response['Body'].read())

print("\n✅ Response:")
print("=" * 60)
print(result[0]["generated_text"])
print("=" * 60)

## 8. Clean Up Resources

**Important**: Delete the endpoint when you're done testing to avoid charges.

Serverless endpoints scale to zero when idle, so they don't incur charges when not processing requests. However, it's still good practice to delete the endpoint when completely finished.

In [ ]:
# Delete the endpoint
print(f"🗑️  Deleting endpoint: {endpoint_name}")

sm_client.delete_endpoint(EndpointName=endpoint_name)
print("✓ Endpoint deleted")

# Optionally delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print("✓ Endpoint configuration deleted")

# Optionally delete model
sm_client.delete_model(ModelName=model_name)
print("✓ Model deleted")

print("\n✅ All resources cleaned up successfully!")

## Next Steps

### Using the Endpoint from Other Applications

You can invoke this endpoint from any AWS SDK or CLI:

```python
import boto3
import json

runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName='your-endpoint-name',
    ContentType='application/json',
    Body=json.dumps({
        "inputs": "Your prompt here",
        "parameters": {
            "max_new_tokens": 128,
            "temperature": 0.7
        }
    })
)

result = json.loads(response['Body'].read())
print(result[0]['generated_text'])
```

### Cost Monitoring

- Serverless endpoints are billed per inference duration (per millisecond)
- No charge when idle (scales to zero)
- Typical cost: ~$0.00002 per second of inference with 3GB memory
- Monitor costs in AWS Cost Explorer under SageMaker > Inference

### Model Optimization

For production use, consider:
- Fine-tuning the model on your specific use case
- Adjusting `max_concurrency` based on expected traffic
- Increasing memory if encountering OOM errors
- Using quantization for larger models (already applied in this config)